In [171]:
%load_ext autoreload
%autoreload 2
import numpy as np
import gym
from keras.callbacks import TensorBoard
from keras.layers import Dense, LSTM, ConvLSTM2D, Activation, Flatten, Conv2D, MaxPooling2D, Input, Lambda
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from collections import deque
from tqdm import tqdm_notebook as tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
total_episodes = 20000
learning_rate = 10e-3
discount_rate = 0.99
batch_size = 64

# Exploitation vs Exploration params.
exploration_probability = 1
max_exploration_probability = 1
min_exploration_probability = 0.01
exploration_probability_decay_rate = 0.995

# Game Actions:
move_up = 2
move_down = 3
possible_actions = [move_up, move_down]
action_space = {2:0, 3:1}

# Image params
image_size = (80,80,1)
memory_size = 10000

In [193]:
def dqn(input_shape, num_actions, loss):
    model = Sequential()
    
    model.add(Dense(16, input_shape=input_shape))
    model.add(Activation("tanh"))
    
    model.add(Dense(16))
    model.add(Activation("tanh"))
    
    model.add(Dense(num_actions))
    
#     optimizer = Adam(lr=0.01, decay=0.01)
    model.compile(loss=loss, optimizer="adam")
    return model

class ReplayMemory:
    def __init__(self, max_replay_memory_size):
        self.memory = deque(maxlen = max_replay_memory_size)
            
    def add_memory(self, current_state, reward, action, next_state, done):
        self.memory.append([current_state, reward, action, next_state, done])
    
    def sample_memory(self, batch_size):
        samples = np.random.choice(np.arange(len(self.memory)), size = batch_size, replace=False)
        return np.array([self.memory[sample] for sample in samples])
#         current_states, rewards, actions, next_states, dones = zip(*[self.memory[sample] for sample in samples])
#         return np.array(current_states), np.array(rewards), np.array(actions), np.array(next_states), np.array(dones)

class Agent:
    def __init__(self, environment, memory_size, exploration_probability):
        ####
        self.loss = "mse"
        self.optimizer = "adam"
        self.exploration_probability = exploration_probability
        self.l = {"loss":1}
        
        ####
        self.env = environment
        self.memory = ReplayMemory(memory_size)
        
        self.policy_network = dqn((4,), 2, self.loss)
        self.target_network = dqn((4,), 2, self.loss)
        self.reset()
    
    def reset(self):
        self.current_state = self.process_state(self.env.reset())
        
    def process_state(self, state):
        return np.reshape(state, [1,4])
    
    def play_step(self):
        if np.random.uniform(0,1) < self.exploration_probability:
            action = self.env.action_space.sample()
        else:
            action = self.policy_network.predict(self.current_state)
            action = np.argmax(action)
        
        next_state, reward, done, _ = self.env.step(action)
        next_state = self.process_state(next_state)
        
        return next_state, reward, done, action
            
    def replay(self, batch_size, episode):
        # Add Target network in later.
        if batch_size > len(self.memory.memory):
            return None
        
        minibatch = self.memory.sample_memory(batch_size)
        
        x_train, y_train = [],[]
        for state, reward, action, next_state, done in minibatch:
            target = self.policy_network.predict(state)
            
            target[0][action] = reward
            if not done:
                target[0][action] = reward + (discount_rate * np.amax(self.target_network.predict(next_state)[0]))
            
            x_train.append(state[0])
            y_train.append(target[0])
            
        self.policy_network.fit(np.array(x_train), np.array(y_train), batch_size=batch_size, verbose=0)
        
        if self.exploration_probability > min_exploration_probability:
            self.exploration_probability *= exploration_probability_decay_rate 
            
    def update_target_network(self):
        self.target_network.set_weights(self.policy_network.get_weights())
        

In [194]:
environment = gym.make('CartPole-v0')
agent = Agent(environment, memory_size, exploration_probability)

In [ ]:
scores = deque(maxlen=100)

for episode in tqdm(range(total_episodes)):
    agent.reset()
    done = False
    score = 0
    while not done:
        next_state, reward, done, action = agent.play_step()
        agent.memory.add_memory(agent.current_state, reward, action, next_state, done)
        agent.current_state = next_state
        score += 1
        
        if done:
            agent.replay(batch_size, episode)
        
        if episode % 20 == 0:
            agent.update_target_network()
            
    scores.append(score)
    print("Episode: {}, Mean-Score: {}, Exploration-Prob: {}".format(episode, np.mean(scores), agent.exploration_probability))


Episode: 0, Mean-Score: 20.0, Exploration-Prob: 1
Episode: 1, Mean-Score: 26.0, Exploration-Prob: 1
Episode: 2, Mean-Score: 21.333333333333332, Exploration-Prob: 0.995
Episode: 3, Mean-Score: 23.5, Exploration-Prob: 0.990025
Episode: 4, Mean-Score: 24.2, Exploration-Prob: 0.985074875
Episode: 5, Mean-Score: 25.666666666666668, Exploration-Prob: 0.9801495006250001
Episode: 6, Mean-Score: 24.428571428571427, Exploration-Prob: 0.9752487531218751
Episode: 7, Mean-Score: 23.25, Exploration-Prob: 0.9703725093562657
Episode: 8, Mean-Score: 22.77777777777778, Exploration-Prob: 0.9655206468094844
Episode: 9, Mean-Score: 22.3, Exploration-Prob: 0.960693043575437
Episode: 10, Mean-Score: 21.636363636363637, Exploration-Prob: 0.9558895783575597
Episode: 11, Mean-Score: 21.416666666666668, Exploration-Prob: 0.9511101304657719
Episode: 12, Mean-Score: 21.153846153846153, Exploration-Prob: 0.946354579813443
Episode: 13, Mean-Score: 20.428571428571427, Exploration-Prob: 0.9416228069143757
Episode: 14,

Episode: 107, Mean-Score: 17.76, Exploration-Prob: 0.5878229785513479
Episode: 108, Mean-Score: 17.67, Exploration-Prob: 0.5848838636585911
Episode: 109, Mean-Score: 17.59, Exploration-Prob: 0.5819594443402982
Episode: 110, Mean-Score: 17.56, Exploration-Prob: 0.5790496471185967
Episode: 111, Mean-Score: 17.47, Exploration-Prob: 0.5761543988830038
Episode: 112, Mean-Score: 17.48, Exploration-Prob: 0.5732736268885887
Episode: 113, Mean-Score: 17.49, Exploration-Prob: 0.5704072587541458
Episode: 114, Mean-Score: 17.64, Exploration-Prob: 0.567555222460375
Episode: 115, Mean-Score: 17.53, Exploration-Prob: 0.5647174463480732
Episode: 116, Mean-Score: 17.68, Exploration-Prob: 0.5618938591163328
Episode: 117, Mean-Score: 17.6, Exploration-Prob: 0.5590843898207511
Episode: 118, Mean-Score: 17.63, Exploration-Prob: 0.5562889678716474
Episode: 119, Mean-Score: 17.63, Exploration-Prob: 0.5535075230322891
Episode: 120, Mean-Score: 17.58, Exploration-Prob: 0.5507399854171277
Episode: 121, Mean-Sco

Episode: 224, Mean-Score: 12.56, Exploration-Prob: 0.326999438029567
Episode: 225, Mean-Score: 12.36, Exploration-Prob: 0.3253644408394192
Episode: 226, Mean-Score: 12.41, Exploration-Prob: 0.3237376186352221
Episode: 227, Mean-Score: 12.47, Exploration-Prob: 0.322118930542046
Episode: 228, Mean-Score: 12.46, Exploration-Prob: 0.32050833588933575
Episode: 229, Mean-Score: 12.36, Exploration-Prob: 0.31890579420988907
Episode: 230, Mean-Score: 12.34, Exploration-Prob: 0.3173112652388396
Episode: 231, Mean-Score: 12.31, Exploration-Prob: 0.3157247089126454
Episode: 232, Mean-Score: 12.18, Exploration-Prob: 0.3141460853680822
Episode: 233, Mean-Score: 12.15, Exploration-Prob: 0.3125753549412418
Episode: 234, Mean-Score: 12.16, Exploration-Prob: 0.31101247816653554
Episode: 235, Mean-Score: 12.1, Exploration-Prob: 0.30945741577570285
Episode: 236, Mean-Score: 12.09, Exploration-Prob: 0.3079101286968243
Episode: 237, Mean-Score: 12.07, Exploration-Prob: 0.3063705780533402
Episode: 238, Mean-

Episode: 342, Mean-Score: 15.28, Exploration-Prob: 0.18099664897669618
Episode: 343, Mean-Score: 15.35, Exploration-Prob: 0.1800916657318127
Episode: 344, Mean-Score: 15.38, Exploration-Prob: 0.17919120740315364
Episode: 345, Mean-Score: 15.48, Exploration-Prob: 0.17829525136613786
Episode: 346, Mean-Score: 15.59, Exploration-Prob: 0.17740377510930716
Episode: 347, Mean-Score: 15.63, Exploration-Prob: 0.17651675623376062
Episode: 348, Mean-Score: 15.73, Exploration-Prob: 0.1756341724525918
Episode: 349, Mean-Score: 15.82, Exploration-Prob: 0.17475600159032884
Episode: 350, Mean-Score: 15.85, Exploration-Prob: 0.17388222158237718
Episode: 351, Mean-Score: 15.91, Exploration-Prob: 0.1730128104744653
Episode: 352, Mean-Score: 15.99, Exploration-Prob: 0.17214774642209296
Episode: 353, Mean-Score: 16.03, Exploration-Prob: 0.1712870076899825
Episode: 354, Mean-Score: 16.09, Exploration-Prob: 0.17043057265153258
Episode: 355, Mean-Score: 16.33, Exploration-Prob: 0.16957841978827493
Episode: 3

Episode: 458, Mean-Score: 28.69, Exploration-Prob: 0.10119240105273684
Episode: 459, Mean-Score: 28.54, Exploration-Prob: 0.10068643904747315
Episode: 460, Mean-Score: 28.47, Exploration-Prob: 0.10018300685223579
Episode: 461, Mean-Score: 28.22, Exploration-Prob: 0.0996820918179746
Episode: 462, Mean-Score: 28.06, Exploration-Prob: 0.09918368135888474
Episode: 463, Mean-Score: 27.85, Exploration-Prob: 0.09868776295209031
Episode: 464, Mean-Score: 27.58, Exploration-Prob: 0.09819432413732986
Episode: 465, Mean-Score: 27.3, Exploration-Prob: 0.09770335251664321
Episode: 466, Mean-Score: 27.1, Exploration-Prob: 0.09721483575406
Episode: 467, Mean-Score: 26.81, Exploration-Prob: 0.09672876157528969
Episode: 468, Mean-Score: 26.61, Exploration-Prob: 0.09624511776741324
Episode: 469, Mean-Score: 26.43, Exploration-Prob: 0.09576389217857617
Episode: 470, Mean-Score: 26.26, Exploration-Prob: 0.09528507271768329
Episode: 471, Mean-Score: 26.1, Exploration-Prob: 0.09480864735409487
Episode: 472,

...

Episode: 12961, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12962, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12963, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12964, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12965, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12966, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12967, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12968, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12969, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12970, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12971, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12972, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12973, Mean-Score: 109.85, Exploration-Prob: 0.00998645168764533
Episode: 12974, Mean-Score: 109.85, Ex

Episode: 13072, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13073, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13074, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13075, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13076, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13077, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13078, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13079, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13080, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13081, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13082, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13083, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13084, Mean-Score: 188.73, Exploration-Prob: 0.00998645168764533
Episode: 13085, Mean-Score: 188.73, Ex

Episode: 13184, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13185, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13186, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13187, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13188, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13189, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13190, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13191, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13192, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13193, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13194, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13195, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13196, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13197, Mean-Score: 200.0, Exploration-Prob

Episode: 13298, Mean-Score: 199.36, Exploration-Prob: 0.00998645168764533
Episode: 13299, Mean-Score: 199.36, Exploration-Prob: 0.00998645168764533
Episode: 13300, Mean-Score: 198.79, Exploration-Prob: 0.00998645168764533
Episode: 13301, Mean-Score: 198.07, Exploration-Prob: 0.00998645168764533
Episode: 13302, Mean-Score: 197.47, Exploration-Prob: 0.00998645168764533
Episode: 13303, Mean-Score: 197.47, Exploration-Prob: 0.00998645168764533
Episode: 13304, Mean-Score: 197.47, Exploration-Prob: 0.00998645168764533
Episode: 13305, Mean-Score: 197.47, Exploration-Prob: 0.00998645168764533
Episode: 13306, Mean-Score: 197.47, Exploration-Prob: 0.00998645168764533
Episode: 13307, Mean-Score: 196.84, Exploration-Prob: 0.00998645168764533
Episode: 13308, Mean-Score: 196.2, Exploration-Prob: 0.00998645168764533
Episode: 13309, Mean-Score: 196.2, Exploration-Prob: 0.00998645168764533
Episode: 13310, Mean-Score: 195.49, Exploration-Prob: 0.00998645168764533
Episode: 13311, Mean-Score: 194.78, Expl

Episode: 13409, Mean-Score: 196.79, Exploration-Prob: 0.00998645168764533
Episode: 13410, Mean-Score: 197.5, Exploration-Prob: 0.00998645168764533
Episode: 13411, Mean-Score: 198.21, Exploration-Prob: 0.00998645168764533
Episode: 13412, Mean-Score: 198.81, Exploration-Prob: 0.00998645168764533
Episode: 13413, Mean-Score: 199.44, Exploration-Prob: 0.00998645168764533
Episode: 13414, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13415, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13416, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13417, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13418, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13419, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13420, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13421, Mean-Score: 200.0, Exploration-Prob: 0.00998645168764533
Episode: 13422, Mean-Score: 200.0, Exploration-